In [1]:
import pickle, random
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.cluster import DBSCAN
import traj_dist.distance as dist


In [2]:
indices = np.arange(10000)

In [3]:
with open('testgps', 'rb') as f:
    gps = pickle.load(f)

gpsp = np.array(gps)[indices]


/tmp/ipykernel_1556022/1349570022.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  gpsp = np.array(gps)[indices]


In [4]:
with open('dist_real_huge', 'rb') as f:
    dist_real = pickle.load(f)

In [5]:
# 通过实际dist_real 计算 sim_real
sim_real = 1 - dist_real / np.max(dist_real)


### 地标
[520, 1607, 5819, 4532, 4865, 2892, 4064, 3132, 6772,  111] -> 3100

[ 520, 1607, 5819, 4532, 4865, 2892, 4064, 3132, 6772,  111,  934,
       6846, 1260, 9212, 2981, 8277, 5134, 7147, 1749, 4889, 7480, 1408,
       7337, 9087,  116, 8246, 9879, 9676,  575, 5135, 7524, 7112, 4617,
       7545, 7044, 3436, 2078, 3646, 7454, 9890, 3433, 4991, 9481, 4300,
       6127, 3709, 1040,  358, 1875, 9859, 3659,  207, 1798,  370, 3712,
       4064, 8074, 9109, 8033, 4096, 8370, 7214, 1601, 2582, 2332,   13,
       7711, 6283, 9411, 3226, 4419, 8929, 2256, 6742, 9780, 6196, 7766,
       5404, 2441, 5995, 2984, 8439, 1953, 3968, 1609, 8375, 5831, 9180,
       2557, 7392, 5979, 1374, 5612, 4847, 9928, 1695, 7706, 7572, 5490,
       9634, 1982, 4873]（100个）-> 3400



In [6]:
landmarks = np.random.choice(10000, size=5, replace=False)
landmarks

array([4952,  649, 8473, 2133, 6114])

In [7]:
# landmarks = np.array([6961, 2134, 3805, 6813, 2615, 3879, 3342, 1574])
landmarks = np.array([520, 1607, 5819, 4532, 4865, 2892, 4064, 3132, 6772,  111])
landmarks

array([ 520, 1607, 5819, 4532, 4865, 2892, 4064, 3132, 6772,  111])

In [8]:
coord_real = np.array([[sim_real[i, j] for j in landmarks] for i in range(10000)])
coord_real[landmarks[1]]

array([0.50025616, 1.        , 0.73000555, 0.78435558, 0.82798452,
       0.74092873, 0.90024937, 0.81756007, 0.83754597, 0.85239344])

In [9]:
with open('128testEmbeddingNeutraj25', 'rb') as f:
    embed = pickle.load(f)
embedp = np.array(embed)[indices]    # indices 等价于 slice[0:10000]


# # Compute distance matrix using discrete frechet distance
# dist_embed = np.zeros((len(embedp), len(embedp)))
# for i in range(len(embedp)):
#     for j in range(i, len(embedp)):
#         a, b = embedp[i], embedp[j]
#         distance = np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
#         dist_embed[i, j] = distance
#         dist_embed[j, i] = distance
# with open('dist_embed', 'wb') as f:
#     pickle.dump(dist_embed_128, f)

with open('dist_embed_huge_128', 'rb') as f:
    dist_embed = pickle.load(f)

In [10]:
coord_embed = [[dist_embed[i, j] for j in landmarks] for i in range(10000)]


In [11]:
import numpy as np
print(np.shape(coord_real), np.shape(coord_embed))

(10000, 10) (10000, 10)


In [12]:
# 为每条轨迹计算landmark的target
target = []
for i in range(10000):
    diff = coord_real[i] - coord_embed
    cost = np.array([np.sqrt(np.sum(diff[j]*diff[j])) for j in range(10000)])
    target.append(np.argmin(cost))
    if (i%1000 == 0):
        print('done{}'.format(i))


done0
done1000
done2000
done3000
done4000
done5000
done6000
done7000
done8000
done9000


In [13]:
# 计算平均距离
ave_dist = 0
dist_mat = []
for i in range(10000):
    ave_dist += dist_real[i, target[i]]*111111
    dist_mat.append([i, dist_real[i, target[i]]*111111])
ave_dist /= 10000
ave_dist

3410.3476257001266

In [14]:
sum([1 for [i,j] in dist_mat if j < 3000])

4715

In [15]:
dist_1k = [i for [i,j] in dist_mat if j < 1000]    #距离小于1000的轨迹下标组成的列表，以此类推
dist_2k = [i for [i,j] in dist_mat if 1000 < j < 2000]
dist_3k = [i for [i,j] in dist_mat if 2000 < j < 3000]
dist_5k = [i for [i,j] in dist_mat if 3000 < j < 5000]
dist_10k = [i for [i,j] in dist_mat if 5000 < j < 10000]

In [16]:
import numpy as np
print(np.shape(dist_1k))

(418,)


In [17]:
import folium
num = 10
my_map = folium.Map(location = [gps[1000][0][1], gps[1000][-1][0]], zoom_start = 15, 
                    tiles='https://mt.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', attr='default')#, tiles = 'https://{s}.tile-cyclosm.openstreetmap.fr/cyclosm/{z}/{x}/{y}.png', attr= 'default')
for t in dist_1k[:num]:
    for i in range(len(gps[indices[t]])):
        folium.Circle( radius = 15,
                                location=[gps[indices[t]][i][1],gps[indices[t]][i][0]],
                                color='blue',
                                opacity = 0.9,
                                fill=True,
                                fill_opacity = 0.9
                                ).add_to(my_map)
    folium.PolyLine(gps[indices[t]][:,[1,0]], color="blue",lineCap='round', weight=5, opacity=0.6).add_to(my_map)

for t in dist_2k[:num]:
    for i in range(len(gps[indices[t]])):
        folium.Circle( radius = 15,
                                location=[gps[indices[t]][i][1],gps[indices[t]][i][0]],
                                color='green',
                                opacity = 1,
                                fill=True,
                                fill_opacity = 1
                                ).add_to(my_map)
    folium.PolyLine(gps[indices[t]][:,[1,0]], color="green",lineCap='round', weight=5, opacity=0.7).add_to(my_map)

for t in dist_3k[:num]:
    for i in range(len(gps[indices[t]])):
        folium.Circle( radius = 15,
                                location=[gps[indices[t]][i][1],gps[indices[t]][i][0]],
                                color='red',
                                opacity = 0.9,
                                fill=True,
                                fill_opacity = 0.9
                                ).add_to(my_map)
    folium.PolyLine(gps[indices[t]][:,[1,0]], color="red",lineCap='round', weight=5, opacity=0.6).add_to(my_map)
for t in dist_5k[:num]:
    for i in range(len(gps[t])):
        folium.Circle( radius = 15,
                            location=[gps[t][i][1],gps[t][i][0]],
                            color='yellow',
                            opacity = 1,
                            fill=True,
                            fill_opacity = 1
                            ).add_to(my_map)

    folium.PolyLine(gps[t][:,[1,0]], color="yellow",lineCap='round', weight=5, opacity=0.6).add_to(my_map)

# folium.Marker(trg_trj[0], popup='<b>Starting Point</b>').add_to(my_map)
# my_map.save('match.html')
my_map

In [18]:
with open('match_results', 'wb') as f:
    pickle.dump(target, f)

In [19]:
match_gps = []
for i in range(10000):
    match_gps.append(gps[target[i]])

In [20]:
with open('match_gps', 'wb') as f:
    pickle.dump(match_gps, f)

In [21]:
# 下面就是极其抽象的第二段代码，主要表现就是缺俩变量
# 破案了，这之后都是不需要的

In [22]:
import folium
my_map = folium.Map(location = [gps[1000][0][1], gps[1000][-1][0]], zoom_start = 15, 
                    tiles='https://mt.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', attr='default')#, tiles = 'https://{s}.tile-cyclosm.openstreetmap.fr/cyclosm/{z}/{x}/{y}.png', attr= 'default')
for t in cnt_de[:num]:
    if t not in cnt_dr[:num]:
        for i in range(len(gps[indices[t]])):
            folium.Circle( radius = 25,
                                location=[gps[indices[t]][i][1],gps[indices[t]][i][0]],
                                color='blue',
                                opacity = 0.9,
                                fill=True,
                                fill_opacity = 0.9
                                ).add_to(my_map)
        folium.PolyLine(gps[indices[t]][:,[1,0]], color="blue",lineCap='round', weight=5, opacity=0.6).add_to(my_map)
    else:
        for i in range(len(gps[indices[t]])):
            folium.Circle( radius = 25,
                                location=[gps[indices[t]][i][1],gps[indices[t]][i][0]],
                                color='green',
                                opacity = 1,
                                fill=True,
                                fill_opacity = 1
                                ).add_to(my_map)
        folium.PolyLine(gps[indices[t]][:,[1,0]], color="green",lineCap='round', weight=5, opacity=0.7).add_to(my_map)

for t in cnt_dr[:50]:
    if t not in cnt_de[:num]:
        for i in range(len(gps[indices[t]])):
            folium.Circle( radius = 25,
                                location=[gps[indices[t]][i][1],gps[indices[t]][i][0]],
                                color='red',
                                opacity = 0.9,
                                fill=True,
                                fill_opacity = 0.9
                                ).add_to(my_map)
        folium.PolyLine(gps[indices[t]][:,[1,0]], color="red",lineCap='round', weight=5, opacity=0.6).add_to(my_map)
# for t in [277,544,488,692,634,135,509,849,242,204,635,163,860,491,958,667,529,527,23,128]:
#     for i in range(len(gps[t])):
#         folium.Circle( radius = 20,
#                             location=[gps[t][i][1],gps[t][i][0]],
#                             color='green',
#                             opacity = 1,
#                             fill=True,
#                             fill_opacity = 1
#                             ).add_to(my_map)

#     folium.PolyLine(gps[t][:,[1,0]], color="green",lineCap='round', weight=4, opacity=0.6).add_to(my_map)

# folium.Marker(trg_trj[0], popup='<b>Starting Point</b>').add_to(my_map)
# my_map.save('match.html')
my_map

NameError: name 'cnt_de' is not defined

In [23]:
traj_e = cnt_de[:num]
traj_r = cnt_dr[:num]


NameError: name 'cnt_de' is not defined

In [ ]:
total = 0
for i in traj_r:
    if i in traj_e:
        total += 1
print(total)

In [ ]:
# 第二段代码， seg-2

def simcnt(trajs, k, dist, cnt):
    neark = []
    for i in range(len(trajs)):
        disti = [dist[trajs[i]][trajs[j]] for j in range(len(trajs))]
        neark.append([disti.index(sorted(disti)[::-1][i]) for i in range(0, k)])
    indexk = [set([trajs[neark[j][i]] for i in range(k)]) for j in range(len(trajs))]
    for i in indexk:
        for j in i:
            cnt[j] += 1
    return cnt

In [ ]:
# 第二段代码， seg-3

def sort_with_indices(lst):
    sorted_lst = sorted(enumerate(lst), key=lambda x: x[1])
    return [index for index, _ in sorted_lst], [value for _, value in sorted_lst]

In [ ]:
cnt_e = [0 for _ in range(len(gpsp))]
cnt_r = [0 for _ in range(len(gpsp))]

for k in range(3, 15):
    simcnt(traj_e, k, -dist_embed, cnt_e)
    simcnt(traj_r, k, dist_real, cnt_r)

indices_e = sort_with_indices([-i for i in cnt_e])[0][:num]
indices_r = sort_with_indices([-i for i in cnt_r])[0][:num]

numr = 0
tr = []
for t in traj_r:
    if t not in indices_r:
        numr += 1
        tr.append(t)
indices_r[(num-numr):] = tr
nume = 0
te = []
for t in traj_e:
    if t not in indices_e:
        nume += 1
        te.append(t)
indices_e[(num-nume):] = te

print(indices_e)
print(indices_r)


In [ ]:
pairs = [[indices_e[i], indices_r[i]] for i in range(num)]
err = 0
suc = 0
for p in pairs:
    err += dist_real[p[0], p[1]]
    if p[0] == p[1]:
        suc += 1
err = err/num*111111
print(err, suc)

In [ ]:

ave_diff = 0
for i in range(num):
    if indices_e[i] in cnt_dr[:real_num]:
        ave_diff += abs(indices_r.index(indices_e[i]) - i)
ave_diff /= total
print(total, ave_diff)

In [ ]:

ave_diff = 0
for i in range(num):
    if indices_e[i] in cnt_dr[:real_num]:
        ave_diff += abs(indices_r.index(indices_e[i]) - i)
ave_diff /= total
print(total, ave_diff)

In [ ]:

ave_diff = 0
for i in range(num):
    if indices_e[i] in cnt_dr[:real_num]:
        ave_diff += abs(indices_r.index(indices_e[i]) - i)
ave_diff /= total
print(total, ave_diff)

In [ ]:
# 第二段代码， seg-1

import math
# 删去预测序列中的非hotcell
def del_zero(trj):
    while 0 in trj:
        trj.remove(0)
    while 1 in trj:
        trj.remove(1)
    while 2 in trj:
        trj.remove(2)
    while 3 in trj:
        trj.remove(3)
    return trj

# 实现由hotcell坐标到gps坐标的转换
def cell2coord(region, cell):
    yoffset = cell // region['numx']
    xoffset = cell % region['numx']
    y = region['miny'] + (yoffset + 0.5) * region['ystep']
    x = region['minx'] + (xoffset + 0.5) * region['xstep']
    return x, y


def meters2lonlat(x, y):
    semimajoraxis = 6378137.0
    lon = x / semimajoraxis / 0.017453292519943295
    t = math.exp(y / 3189068.5)
    lat = math.asin((t - 1) / (t + 1)) / 0.017453292519943295
    return lon, lat

def cell2gps(region, cell):
    x, y = cell2coord(region, cell)
    lon, lat =meters2lonlat(x, y)
    return [lat, lon]   
# 定义Frechet距离

# 这个方法是计算两点的距离公式
def euc_dist(pt1, pt2):
    return math.sqrt((pt2[0]-pt1[0])*(pt2[0]-pt1[0])+(pt2[1]-pt1[1])*(pt2[1]-pt1[1]))
 
# 这个就是计算Frechet Distance距离的具体过程,是用递归方式计算
def _c(ca,i,j,P,Q):
    if ca[i,j] > -1:
        return ca[i,j]
    elif i == 0 and j == 0:
        ca[i,j] = euc_dist(P[0],Q[0])
    elif i > 0 and j == 0:
        ca[i,j] = max(_c(ca,i-1,0,P,Q),euc_dist(P[i],Q[0]))
    elif i == 0 and j > 0:
        ca[i,j] = max(_c(ca,0,j-1,P,Q),euc_dist(P[0],Q[j]))
    elif i > 0 and j > 0:
        ca[i,j] = max(min(_c(ca,i-1,j,P,Q),_c(ca,i-1,j-1,P,Q),_c(ca,i,j-1,P,Q)),euc_dist(P[i],Q[j]))
    else:
        ca[i,j] = float("inf")
    return ca[i,j]
 
# 这个是给我们调用的方法
def frechet_distance(P,Q):
    ca = np.ones((len(P),len(Q)))
    ca = np.multiply(ca,-1)
    return _c(ca, len(P) - 1, len(Q) - 1, P, Q)  # ca是a*b的矩阵(3*4),2,3

# -------------------------------------------------------
# 定义Hausdorff距离

def Hausdorff(A, B):
    
    asize = [len(A),len(A[0])]
    bsize = [len(B),len(B[0])]
    
    if asize[1] != bsize[1]:
        print('The dimensions of points in the two sets are not equal')


    fhd = []
    for i in range(asize[0]):
        mindist = float("inf")
        for j in range(bsize[0]):
            tempdist = np.linalg.norm(A[i]-B[j])
            if tempdist < mindist:
                mindist = tempdist
        fhd.append(mindist)
    
    fhd = max(fhd)

    rhd = []
    for j in range(bsize[0]):
        mindist = float("inf")
        for i in range(asize[0]):
            tempdist = np.linalg.norm(A[i]-B[j])
            if tempdist < mindist:
                mindist = tempdist
        rhd.append(mindist)
    rhd = max(rhd)
    
    mhd = max(fhd,rhd)

    return mhd

# -------------------------------------------------------
# 最小覆盖原求解函数

def make_circle(points):
	# Convert to float and randomize order
	shuffled = [(float(x), float(y)) for (x, y) in points]
	random.shuffle(shuffled)
	
	# Progressively add points to circle or recompute circle
	c = None
	for (i, p) in enumerate(shuffled):
		if c is None or not is_in_circle(c, p):
			c = _make_circle_one_point(shuffled[ : i + 1], p)
	return c


# One boundary point known
def _make_circle_one_point(points, p):
	c = (p[0], p[1], 0.0)
	for (i, q) in enumerate(points):
		if not is_in_circle(c, q):
			if c[2] == 0.0:
				c = make_diameter(p, q)
			else:
				c = _make_circle_two_points(points[ : i + 1], p, q)
	return c


# Two boundary points known
def _make_circle_two_points(points, p, q):
	circ = make_diameter(p, q)
	left  = None
	right = None
	px, py = p
	qx, qy = q
	
	# For each point not in the two-point circle
	for r in points:
		if is_in_circle(circ, r):
			continue
		
		# Form a circumcircle and classify it on left or right side
		cross = _cross_product(px, py, qx, qy, r[0], r[1])
		c = make_circumcircle(p, q, r)
		if c is None:
			continue
		elif cross > 0.0 and (left is None or _cross_product(px, py, qx, qy, c[0], c[1]) > _cross_product(px, py, qx, qy, left[0], left[1])):
			left = c
		elif cross < 0.0 and (right is None or _cross_product(px, py, qx, qy, c[0], c[1]) < _cross_product(px, py, qx, qy, right[0], right[1])):
			right = c
	
	# Select which circle to return
	if left is None and right is None:
		return circ
	elif left is None:
		return right
	elif right is None:
		return left
	else:
		return left if (left[2] <= right[2]) else right


def make_diameter(a, b):
	cx = (a[0] + b[0]) / 2
	cy = (a[1] + b[1]) / 2
	r0 = math.hypot(cx - a[0], cy - a[1])
	r1 = math.hypot(cx - b[0], cy - b[1])
	return (cx, cy, max(r0, r1))


def make_circumcircle(a, b, c):
	# Mathematical algorithm from Wikipedia: Circumscribed circle
	ox = (min(a[0], b[0], c[0]) + max(a[0], b[0], c[0])) / 2
	oy = (min(a[1], b[1], c[1]) + max(a[1], b[1], c[1])) / 2
	ax = a[0] - ox;  ay = a[1] - oy
	bx = b[0] - ox;  by = b[1] - oy
	cx = c[0] - ox;  cy = c[1] - oy
	d = (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by)) * 2.0
	if d == 0.0:
		return None
	x = ox + ((ax*ax + ay*ay) * (by - cy) + (bx*bx + by*by) * (cy - ay) + (cx*cx + cy*cy) * (ay - by)) / d
	y = oy + ((ax*ax + ay*ay) * (cx - bx) + (bx*bx + by*by) * (ax - cx) + (cx*cx + cy*cy) * (bx - ax)) / d
	ra = math.hypot(x - a[0], y - a[1])
	rb = math.hypot(x - b[0], y - b[1])
	rc = math.hypot(x - c[0], y - c[1])
	return (x, y, max(ra, rb, rc))


_MULTIPLICATIVE_EPSILON = 1 + 1e-14

def is_in_circle(c, p):
	return c is not None and math.hypot(p[0] - c[0], p[1] - c[1]) <= c[2] * _MULTIPLICATIVE_EPSILON


# Returns twice the signed area of the triangle defined by (x0, y0), (x1, y1), (x2, y2).
def _cross_product(x0, y0, x1, y1, x2, y2):
	return (x1 - x0) * (y2 - y0) - (y1 - y0) * (x2 - x0)
def lonlat2meters(lon, lat):
    semimajoraxis = 6378137.0
    east = lon * 0.017453292519943295
    north = lat * 0.017453292519943295
    t = np.sin(north)
    return semimajoraxis * east, 3189068.5 * np.log((1 + t) / (1 - t))

In [ ]:
f_distance = []
se_dist = []
h_distance = []
diameter = []

for i in range(num):
    pre_trj = gps[indices[pairs[i][0]]]
    pre_trj = list(map(lambda x: np.asarray(lonlat2meters(x[0], x[1])),pre_trj))
    if (np.linalg.norm(pre_trj[0]-pre_trj[-1])) <= 500:
        i += 1
        continue
    trg_trj = gps[indices[pairs[i][1]]]
    trg_trj = list(map(lambda x: np.asarray(lonlat2meters(x[0], x[1])),trg_trj))
    if (np.linalg.norm(trg_trj[0]-trg_trj[-1])) <= 500:
        i += 1
        continue
    f = frechet_distance(trg_trj, pre_trj)
    

    f_distance.append(f)  # 轨迹Frechet距离
frec_distance = np.asarray(f_distance)
print("Average Frechet distance:",sum(frec_distance)/num,"meters")


In [ ]:
near_3_e = []
near_3_r = []


for i in range(50):
    dist_e = [dist_embed[traj_e[i]][traj_e[j]] for j in range(num)]
    near_3_e.append([dist_e.index(sorted(dist_e)[::-1][i]) for i in range(0, 3)])
    
    dist_r = [dist_real[traj_r[i]][traj_r[j]] for j in range(num)]
    near_3_r.append([dist_r.index(sorted(dist_r)[i]) for i in range(0, 3)])
    

In [ ]:
print([set([traj_e[near_3_e[j][i]] for i in range(3)]) for j in range(num)])
print([set([traj_r[near_3_r[j][i]] for i in range(3)]) for j in range(num)])

In [ ]:
near_4_e = []
near_4_r = []


for i in range(50):
    dist_e = [dist_embed[traj_e[i]][traj_e[j]] for j in range(num)]
    near_4_e.append([dist_e.index(sorted(dist_e)[::-1][i]) for i in range(0, 4)])
    
    dist_r = [dist_real[traj_r[i]][traj_r[j]] for j in range(num)]
    near_4_r.append([dist_r.index(sorted(dist_r)[i]) for i in range(0, 4)])
    

In [ ]:
print([set([traj_e[near_4_e[j][i]] for i in range(4)]) for j in range(num)])
print([set([traj_r[near_4_r[j][i]] for i in range(4)]) for j in range(num)])

In [ ]:
index_3_e = [set([traj_e[near_3_e[j][i]] for i in range(3)]) for j in range(num)]
index_3_r = [set([traj_r[near_3_r[j][i]] for i in range(3)]) for j in range(num)]
index_4_e = [set([traj_e[near_4_e[j][i]] for i in range(4)]) for j in range(num)]
index_4_r = [set([traj_r[near_4_r[j][i]] for i in range(4)]) for j in range(num)]

In [ ]:
num_3 = 0
num_4 = 0
match_3 = []
match_4 = []
for i in range(num):
    if index_3_e[i] in index_3_r:
        num_3 += 1
        match_3.append(index_3_e[i])

    if index_4_e[i] in index_4_r:
        num_4 += 1
        match_4.append(index_4_e[i])

print(num_3, num_4)

In [ ]:
match_3

In [ ]:
for set_4 in match_4:
    for set_3 in match_3:
        if set_3.issubset(set_4):
            print(set_3, set_4)
            break